Tutorial to calculate mask means (regional means) of variables within a Profile object.

Provide paths to four files:

    fn_dom : NEMO domain file defining mask lon/lat.
    fn_cfg_nemo : NEMO config file.
    fn_profile : Path to netCDF containing profile data.
    fn_out : Path to netCDF output file.

You can use this script with example files by setting:
    
    fn_dom = path.join('./example_files', "coast_example_nemo_domain.nc")
    fn_prof = path.join('./example_files', "coast_example_en4_201008.nc")
    fn_cfg_nemo = path.join('./config', "example_nemo_grid_t.json")
    fn_cfg_prof = path.join('./config', "example_en4_profiles.json")

### Import relevant packages

In [ ]:
import coast
import numpy as np
from os import path

### Set filepaths to data and configuration

In [ ]:
"""
fn_dom = "<PATH_TO_NEMO_DOMAIN_FILE>"
fn_cfg_nemo = "<PATH_TO_COAST_GRIDDED_CONFIG_FILE>"
fn_cfg_prof = "<PATH_TO_COAST_PROFILE_CONFIG_FILE>"
fn_prof = "<PATH_TO_COAST_PROFILE_NETCDF>"
fn_out = "<PATH_TO_OUTPUT_FILE>"
"""

fn_out = "./output.nc"
fn_dom = path.join('./example_files', "coast_example_nemo_domain.nc")
fn_prof = path.join('./example_files', "coast_example_en4_201008.nc")
fn_cfg_nemo = path.join('./config', "example_nemo_grid_t.json")
fn_cfg_prof = path.join('./config', "example_en4_profiles.json")

### Create NEMO object and read in NEMO data

In [ ]:
nemo = coast.Gridded(fn_domain=fn_dom, multiple=True, config=fn_cfg_nemo)

### Extract latitude and longitude array

In [ ]:
lon = nemo.dataset.longitude.values.squeeze()
lat = nemo.dataset.latitude.values.squeeze()

### Create analysis object and mask maker object

In [ ]:
profile_analysis = coast.ProfileAnalysis()

### Make Profile object and read data

In [ ]:
profile = coast.Profile(config=fn_cfg_prof)
profile.read_en4(fn_prof)

### Make MaskMaker object and define Regional Masks

In [ ]:
# Make MaskMaker object
mm = coast.MaskMaker()

# Define Regional Masks
regional_masks = []
bath = nemo.dataset.bathymetry.values
regional_masks.append(np.ones(lon.shape))
regional_masks.append(mm.region_def_nws_north_sea(lon, lat, bath))
regional_masks.append(mm.region_def_nws_outer_shelf(lon, lat, bath))
regional_masks.append(mm.region_def_nws_english_channel(lon, lat, bath))
regional_masks.append(mm.region_def_nws_norwegian_trench(lon, lat, bath))
regional_masks.append(mm.region_def_kattegat(lon, lat, bath))
regional_masks.append(mm.region_def_south_north_sea(lon, lat, bath))
off_shelf = mm.region_def_off_shelf(lon, lat, bath)
off_shelf[regional_masks[3].astype(bool)] = 0
off_shelf[regional_masks[4].astype(bool)] = 0
regional_masks.append(off_shelf)
regional_masks.append(mm.region_def_irish_sea(lon, lat, bath))

region_names = [
    "whole_domain",
    "north_sea",
    "outer_shelf",
    "eng_channel",
    "nor_trench",
    "kattegat",
    "southern_north_sea",
    "irish_sea",
    "off_shelf",
]

mask_list = mm.make_mask_dataset(lon, lat, regional_masks)
mask_indices = profile_analysis.determine_mask_indices(profile, mask_list)

### Do mask averaging

In [ ]:
mask_means = profile_analysis.mask_means(profile, mask_indices)

### Save mask dataset to file

In [ ]:
mask_means.to_netcdf(fn_out)